In [1]:
from particle import Particle,PDGID
import uproot
import numpy as np
from IPython.display import display, Latex
import glob
import ROOT
import os
import copy

Welcome to JupyROOT 6.22/06


In [2]:
# Define momentum ranges
momentum_ranges = np.arange(0,5,0.5)
num_ranges = len(momentum_ranges)

In [3]:
# Define plot marker styles and colors
marker_styles = [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
marker_colors = [1]

In [43]:
def get_baryon_list(wildcard):
    # Load the TTree from the ROOT file
    files = glob.glob(wildcard)
    upid_arr = np.array([],dtype=int)
    for file in files:
        print(file)
        file = uproot.open(file)
        tree = file["EventTree"]


        for branch in (["trueparentpid","trueparentparentpid"]):
            pid_arr = tree[branch].array()
            pid_arr = [element for sublist in pid_arr for element in sublist]
            upid_arr = np.unique(np.concatenate([upid_arr,pid_arr]))
    
    baryon_list = []
    baryon_name = []
    for pid in upid_arr:
        if(pid==-999 or pid==91 or pid==92):
            continue
        particle = PDGID(pid)
        PARTICLE = Particle.from_pdgid(pid)
        if particle.is_baryon:
            baryon_list.append(int(pid))
            baryon_name.append(f"${PARTICLE.latex_name}$")
    print("Baryon PIDs:", baryon_list)
    print("Baryon Names:", display(Latex(" , ".join(baryon_name))))
    return baryon_list

In [44]:
baryon_pids=get_baryon_list("/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/volatile/data/*/MC_RGA_3051_*.root")

/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/volatile/data/piplus_piminus/MC_RGA_3051_3.root
/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/volatile/data/piplus_piminus/MC_RGA_3051_2.root
/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/volatile/data/piplus_piminus/MC_RGA_3051_0.root
/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/volatile/data/piplus_piminus/MC_RGA_3051_4.root
/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/volatile/data/piplus_piminus/MC_RGA_3051_1.root
/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/volatile/data/piplus_piplus/MC_RGA_3051_0.root
/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/volatile/data/piplus_piplus/MC_RGA_3051_4.root
/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/volatile/data/piplus_piplus/MC_RGA_3051_1.root
/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/volatile/data/piplus_piplus/MC_RGA_3051_3.r

<IPython.core.display.Latex object>

Baryon Names: None


In [45]:
def get_baryon_cut(reco_pid):
    if(reco_pid==211 or reco_pid==-211):
        variable="trueparentpid"
    elif(reco_pid==22):
        variable="trueparentparentpid"
    cut = "||".join([f"{variable}=={bpid}" for bpid in baryon_pids])
    return cut

In [46]:
# def plot_contamination_baryon(project_dir, channel, version, reco_pid, momentum_ranges):
#     if(not os.path.exists(f"{project_dir}/systematics/baryon_id")):
#         os.makedirs(f"{project_dir}/systematics/baryon_id")
#     if(not os.path.exists(f"{project_dir}/systematics/baryon_id/{channel}")):
#         os.makedirs(f"{project_dir}/systematics/baryon_id/{channel}")
    
#     # Open input file and get TTree
#     if(version=="MC_RGA_inbending"):
#         suffix="MC_RGA_30*.root"
#     elif(version=="MC_RGA_outbending"):
#         suffix="MC_RGA_3313_*.root"
    
#     tree = ROOT.TChain("EventTree")
#     tree.Add(f"{project_dir}/volatile/data/{channel}/{suffix}")
#     CHANNEL=channel.replace("piplus","#pi^{+}").replace("piminus","#pi^{-}").replace("pi0","#pi^{0}").replace("_","")
#     # Define lists to store TH1F objects
#     hist_list = []
#     # Create TCanvas and TLegend
#     canvas = ROOT.TCanvas("canvas", "Baryon Contamination vs. Theta",800,600)
#     legend = ROOT.TLegend(0.4, 0.15, 0.85, 0.4)
#     legend.SetNColumns(2)
#     ROOT.gStyle.SetOptStat(0)
#     # Loop over momentum ranges and create TH1F for each range
#     for i in range(len(momentum_ranges)-1):
#         pmin=momentum_ranges[i]
#         pmax=momentum_ranges[i+1]
#         # Create TH1F for this momentum range
#         hist_false = ROOT.TH1F("hist_false{}".format(i), "Baryon Contamination vs. Theta ( {} dihadrons)".format(CHANNEL), 15, 8, 35)
#         hist_total = ROOT.TH1F("hist_total{}".format(i), "Baryon Contamination vs. Theta ( {} dihadrons)".format(CHANNEL), 15, 8, 35)
#         # Fill histograms with contamination percentages
#         baryon_cut=get_baryon_cut(reco_pid)
#         tree.Draw("theta*180/3.1415 >> hist_false{}".format(i), "pid == {} && ({})  && p > {} && p < {}".format(reco_pid,baryon_cut,pmin, pmax), "goff")
#         tree.Draw("theta*180/3.1415 >> hist_total{}".format(i), "pid == {} && p > {} && p < {}".format(reco_pid,pmin, pmax), "goff")
#         # Divide histograms to get contamination percentage
#         hist_false.Divide(hist_total)
#         # Add TH1F to list
#         hist_list.append(hist_false)
#         # Set histogram style and add to legend
#         hist_list[-1].SetMarkerColor(marker_colors[i%len(marker_colors)])
#         hist_list[-1].SetMarkerStyle(marker_styles[i//len(marker_colors)])
#         hist_list[-1].SetMarkerSize(1.5)
#         legend.AddEntry(hist_list[-1], "{:.1f} < p < {:.1f} GeV".format(pmin, pmax), "p")
#         # Draw TH1F on canvas
#         hist_list[-1].Draw("P same" if i!=0 else "P")
#     # Set canvas properties and draw legend
#     hist_list[0].GetYaxis().SetRangeUser(0.00001,1)
#     canvas.SetLogy(1)
#     canvas.SetGrid()
#     legend.Draw("same")
#     # Set axis labels and draw on canvas
#     hist_list[0].GetXaxis().SetTitle("Theta [deg]")
#     hist_list[0].GetYaxis().SetTitle("Contamination")
#     canvas.SaveAs(f"{project_dir}/systematics/baryon_id/{channel}/figure_{reco_pid}_{version}.png")
    
#     return copy.deepcopy(canvas)

In [50]:
def plot_contamination_baryon(project_dir, channel, version, reco_pid, momentum_ranges):
    def create_directory(path):
        if not os.path.exists(path):
            os.makedirs(path)

    def get_version_suffix(version):
        if version == "MC_RGA_inbending":
            return "MC_RGA_30*.root"
        elif version == "MC_RGA_outbending":
            return "MC_RGA_3313_*.root"

    create_directory(f"{project_dir}/systematics/baryon_id")
    create_directory(f"{project_dir}/systematics/baryon_id/{channel}")

    suffix = get_version_suffix(version)

    tree = ROOT.TChain("EventTree")
    tree.Add(f"{project_dir}/volatile/data/{channel}/{suffix}")
    CHANNEL = channel.replace("piplus", "#pi^{+}").replace("piminus", "#pi^{-}").replace("pi0", "#pi^{0}").replace("_", "")

    hist_list = []
    canvas = ROOT.TCanvas("canvas", "Baryon Contamination vs. Theta", 800, 600)
    legend = ROOT.TLegend(0.4, 0.15, 0.85, 0.4)
    legend.SetNColumns(2)
    ROOT.gStyle.SetOptStat(0)

    for i in range(len(momentum_ranges) - 1):
        pmin = momentum_ranges[i]
        pmax = momentum_ranges[i + 1]

        hist_false = ROOT.TH1F(f"hist_false{i}", f"Baryon Contamination vs. Theta ( {CHANNEL} dihadrons)", 15, 8, 35)
        hist_total = ROOT.TH1F(f"hist_total{i}", f"Baryon Contamination vs. Theta ( {CHANNEL} dihadrons)", 15, 8, 35)

        baryon_cut = get_baryon_cut(reco_pid)
        tree.Draw(f"theta*180/3.1415 >> hist_false{i}", f"pid == {reco_pid} && ({baryon_cut})  && p > {pmin} && p < {pmax}", "goff")
        tree.Draw(f"theta*180/3.1415 >> hist_total{i}", f"pid == {reco_pid} && p > {pmin} && p < {pmax}", "goff")

        hist_false.Divide(hist_total)
        hist_list.append(hist_false)

        hist_list[-1].SetMarkerColor(marker_colors[i % len(marker_colors)])
        hist_list[-1].SetMarkerStyle(marker_styles[i // len(marker_colors)])
        hist_list[-1].SetMarkerSize(1.5)
        legend.AddEntry(hist_list[-1], f"{pmin:.1f} < p < {pmax:.1f} GeV", "p")

        hist_list[-1].Draw("P same" if i != 0 else "P")

    hist_list[0].GetYaxis().SetRangeUser(0.00001, 1)
    canvas.SetLogy(1)
    canvas.SetGrid()
    legend.Draw("same")

    hist_list[0].GetXaxis().SetTitle("Theta [deg]")
    hist_list[0].GetYaxis().SetTitle("Contamination")
    canvas.SaveAs(f"{project_dir}/systematics/baryon_id/{channel}/figure_{reco_pid}_{version}.png")

    return copy.deepcopy(canvas)

In [51]:
project_dir="/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0"
versions = ["MC_RGA_inbending","MC_RGA_outbending"]
channels = ["piplus_piminus", "piplus_pi0", "piplus_piplus","pi0_pi0","piminus_pi0", "piminus_piminus"]

In [52]:
def get_reco_pids(channel):
    reco_pid=[]
    if("piplus" in channel):
        reco_pid.append(211)
    if("piminus" in channel):
        reco_pid.append(-211)
    if("pi0" in channel):
        reco_pid.append(22)
    return reco_pid

In [53]:
for version in versions:
    for channel in channels:
        reco_pids=get_reco_pids(channel)
        for rp in reco_pids:
            print(version,channel,"\t pid=",rp)
            plot_contamination_baryon(project_dir,channel,version,rp,momentum_ranges)

MC_RGA_inbending piplus_piminus 	 pid= 211
MC_RGA_inbending piplus_piminus 	 pid= -211
MC_RGA_inbending piplus_pi0 	 pid= 211
MC_RGA_inbending piplus_pi0 	 pid= 22
MC_RGA_inbending piplus_piplus 	 pid= 211
MC_RGA_inbending pi0_pi0 	 pid= 22
MC_RGA_inbending piminus_pi0 	 pid= -211
MC_RGA_inbending piminus_pi0 	 pid= 22
MC_RGA_inbending piminus_piminus 	 pid= -211
MC_RGA_outbending piplus_piminus 	 pid= 211
MC_RGA_outbending piplus_piminus 	 pid= -211
MC_RGA_outbending piplus_pi0 	 pid= 211
MC_RGA_outbending piplus_pi0 	 pid= 22
MC_RGA_outbending piplus_piplus 	 pid= 211
MC_RGA_outbending pi0_pi0 	 pid= 22
MC_RGA_outbending piminus_pi0 	 pid= -211
MC_RGA_outbending piminus_pi0 	 pid= 22
MC_RGA_outbending piminus_piminus 	 pid= -211


Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/baryon_id/piplus_piminus/figure_211_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/baryon_id/piplus_piminus/figure_-211_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/baryon_id/piplus_pi0/figure_211_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/baryon_id/piplus_pi0/figure_22_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/baryon_id/piplus_piplus/figure_211_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/p

In [54]:
import os

# Define the output filename for the LaTeX file
latex_filename = f"{project_dir}/systematics/baryon_id/plots.tex"

# Open the LaTeX file for writing
with open(latex_filename, "w") as f:
    # Write the LaTeX document preamble
    f.write("\\documentclass{article}\n")
    f.write("\\usepackage{graphicx}\n")
    #f.write("\\usepackage{caption}\n")
    f.write("\\usepackage{geometry}\n")
    f.write("\\usepackage{fancyhdr}\n")

    f.write("\\geometry{margin=2cm}\n") # Set the margin to 1 cm
    f.write("\\usepackage[font=large]{caption}\n") # Set the caption font size to large
    f.write("\\begin{document}\n")
    f.write("\\pagestyle{fancy}\n")
    f.write("\\fancyhf{}\n")
    f.write("\\fancyhead[L]{\\leftmark\\,\\,\\,\\,\\rightmark}\n")
    f.write("\\fancyhead[R]{\\thepage}\n")
    # Add a table of contents
    f.write("\\tableofcontents\n")
    # Loop over all channels
    for channel in channels:
        CHANNEL=channel.replace("piplus","$\\pi^{+}$").replace("piminus","$\\pi^{-}$").replace("pi0","$\\pi^{0}$").replace("_","")
        f.write(f"\\section{{{CHANNEL} dihadrons}}") # Add a section for the channel
        
        # Loop over all versions
        for version in versions:
            VERSION=version.replace("_"," ")
            f.write(f"\\subsection{{{VERSION}}}") # Add a subsection for the version
            reco_pids=get_reco_pids(channel)
            # Loop over all binstructs
            for rp in reco_pids:
                # Define the plot filename and caption
                plotname = f'{project_dir}/systematics/baryon_id/{channel}/figure_{rp}_{version}.png'
                parent_string = "Parent" if rp!=22 else "Grandparent"
                if(rp==22):
                    pid_string="$\\gamma$"
                elif(rp==211):
                    pid_string="$\\pi^{+}$"
                elif(rp==-211):
                    pid_string="$\\pi^{-}$"
                caption = f"NEWLINE{parent_string} Baryon$\\rightarrow${pid_string}NEWLINEMonte Carlo Version={version}"
                #plotname = plotname.replace("_","\_")
                caption = caption.replace("_","\_").replace("NEWLINE","\\\\")
                # Write the LaTeX code for the plot and caption
                f.write("\\begin{figure}[ht]\n")
                f.write(f"\\includegraphics[width=\\textwidth]{{{plotname}}}\n")
                f.write(f"\\captionsetup{{font=large}}\n") # Set the caption font size to large
                f.write(f"\\caption{{{caption}}}\n")
                f.write("\\end{figure}\n")
            f.write("\\clearpage")
    # Write the LaTeX document postamble
    f.write("\\end{document}\n")
    
# Get the directory part of the LaTeX filename
latex_dirname = os.path.dirname(latex_filename)

# Save the current working directory
cwd = os.getcwd()

# Change the current working directory to the LaTeX directory
os.chdir(latex_dirname)

# Compile the LaTeX file into a PDF
os.system(f"pdflatex {latex_filename}")
os.system(f"pdflatex {latex_filename}")

# Change the current working directory back to the original
os.chdir(cwd)

This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode

(/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/ba
ryon_id/plots.tex
LaTeX2e <2020-02-02> patch level 2
L3 programming layer <2020-02-14>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphicx.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphics.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/trig.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics-cfg/graphics.cfg)
(/usr/share/texlive/texmf-dist/tex/latex/graphics-def/pdftex.def)))
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/generic/ift